# Cluebot - Modeling - Bayes-based Methods

In [13]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

In [14]:
import pandas as pd
from feature_engineer import preprocessor

train_data = pd.read_csv('/Users/robin/Documents/GitHub/Cluebot/train_data.csv')
train_data = train_data.reset_index(drop=True)
preprocessor.preprocessor(train_data)

test_data = pd.read_csv('/Users/robin/Documents/GitHub/Cluebot/test_data.csv')

train_data.sample(5)

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person,comment_empty,account_age,is_IP,word_count_added,word_count_deleted
13216,change,229473995,/* Modern astronomers */,93.96.38.200,9,7,0,20080802214649,Jason Patton,NaN,...,1217286609,"""Contrary to the classical image of an old ast...",True,2,0,False,1,True,150,150
14501,change,230012102,/* Filmography */,86.4.18.29,1,1,0,20080805163143,TheYoungDoctor,NaN,...,1216983053,"==Filmography==,",True,0,1,False,1,True,2,1
15284,change,327880970,NaN,Jack1956,102118,15619,2,1179350131,Jack1956,NaN,...,1259165771,"""He appeared in the original [[Broadway]] prod...",False,2,1,True,923,False,51,47
21789,change,230508181,[[WP:AES|←]] Replaced content with '\nYou is a...,4.246.210.114,3,2,2,20080807234526,J.delanoy,NaN,...,1218152552,"{{Refimprove|date=October 2007}},{{OR|date=Mar...",True,8,0,False,1,True,32,543
10893,change,328717972,"/* ""Red Data"" */",Mc53912,45,7,0,1120465072,Mc53912,NaN,...,1259548893,"""In ''X-Men'' #197, While the team rushed Rogu...",False,2,0,False,1609,False,140,139


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB

n_splits = 5

kfold = StratifiedKFold(n_splits,
                           shuffle=True,
                           random_state=498)

features = ['user_edit_count', 'user_warns', 'user_distinct_pages', 'num_recent_edits', 'num_recent_reversions', 'current_minor', \
            'num_edits_5d_before', 'is_person', \
            'is_IP', 'account_age', 'comment_empty', 'word_count_added', 'word_count_deleted', \
            'added_lines', 'deleted_lines', 'EditID']

from feature_engineer import preprocessor

preprocessor.preprocessor(train_data)
# train_data = train_data.reset_index(drop=True)
train_data.sample(5)

from feature_engineer import vandalism_scorer as vs

# scorer = vs.VandalismScorer(n_splits=4, random_state=42)
# scorer.fit(train_data, train_data['isvandalism'])

bayes_accs = np.zeros((n_splits, 3))

for i, (train_index, test_index) in enumerate(kfold.split(train_data, train_data['isvandalism'])):
    edits_tt = train_data.iloc[train_index]
    edits_ho = train_data.iloc[test_index]

    model_pipe = Pipeline([('scorer', vs.VandalismScorer(n_splits = 5)), ('nb', GaussianNB())])

    print(edits_tt.columns)
    
    ## Gaussian Naive Bayes
    model_pipe.fit(edits_tt[features], edits_tt['isvandalism'])
    
    nb_pred = model_pipe.predict(edits_ho[features])
    
    bayes_accs[i, 2] = accuracy_score(edits_ho['isvandalism'], nb_pred)

np.mean(bayes_accs, axis=0)

Index(['EditType', 'EditID', 'comment', 'user', 'user_edit_count',
       'user_distinct_pages', 'user_warns', 'user_reg_time', 'prev_user',
       'common', 'current', 'previous', 'page_made_time', 'title', 'namespace',
       'creator', 'num_recent_edits', 'num_recent_reversions', 'current_minor',
       'current_timestamp', 'added_lines', 'previous_timestamp',
       'deleted_lines', 'isvandalism', 'num_edits_5d_before', 'is_person',
       'comment_empty', 'account_age', 'is_IP', 'word_count_added',
       'word_count_deleted'],
      dtype='object')
Index(['EditType', 'EditID', 'comment', 'user', 'user_edit_count',
       'user_distinct_pages', 'user_warns', 'user_reg_time', 'prev_user',
       'common', 'current', 'previous', 'page_made_time', 'title', 'namespace',
       'creator', 'num_recent_edits', 'num_recent_reversions', 'current_minor',
       'current_timestamp', 'added_lines', 'previous_timestamp',
       'deleted_lines', 'isvandalism', 'num_edits_5d_before', 'is_person'

array([0.       , 0.       , 0.7759339])

In [22]:
# Bayes-based Methods

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

features = ['user_edit_count', 'user_warns', 'user_distinct_pages', 'num_recent_edits', 'num_recent_reversions', 'current_minor', \
            'num_edits_5d_before', 'is_person', \
            'is_IP', 'account_age', 'comment_empty', 'word_count_added', 'word_count_deleted']

bayes_accs = np.zeros((n_splits, 3))

for i, (train_index, test_index) in enumerate(kfold.split(train_data, train_data['isvandalism'])):
    edits_tt = train_data.iloc[train_index]
    edits_ho = train_data.iloc[test_index]
    
    ## Linear Discriminant Analysis
    lda = LinearDiscriminantAnalysis()
    
    lda.fit(edits_tt[features], edits_tt['isvandalism'])
    lda_pred = lda.predict(edits_ho[features])
    
    bayes_accs[i, 0] = accuracy_score(edits_ho['isvandalism'], lda_pred)
    
    ## Quadratic Discriminant Analysis
    qda = QuadraticDiscriminantAnalysis(store_covariance = True)
    
    qda.fit(edits_tt[features], edits_tt['isvandalism'])
    
    qda_pred = qda.predict(edits_ho[features])
    
    bayes_accs[i, 1] = accuracy_score(edits_ho['isvandalism'], qda_pred)
    
    
    ## Gaussian Naive Bayes
    nb = GaussianNB()
    
    nb.fit(edits_tt[features], edits_tt['isvandalism'])
    
    nb_pred = nb.predict(edits_ho[features])
    
    bayes_accs[i, 2] = accuracy_score(edits_ho['isvandalism'], nb_pred)

np.mean(bayes_accs, axis=0)

array([0.81654153, 0.79437394, 0.7759339 ])

In [17]:
import random
import numpy as np
import re
from collections import defaultdict

# Do a cross validation split manually
total_indices = list(range(len(train_data))) # 
random.shuffle(total_indices)
fold = [[], [], [], [], []]
[fold[0], fold[1], fold[2], fold[3], fold[4]] = np.array_split(total_indices, 5)
for i in range(5):
    fold[i] = list(fold[i])

# Get the difference between two strings and return a set of words
def get_difference(s1, s2):
    w1 = set(re.sub(r"[^\w\s]", " ", str(s1)).lower().split())
    w2 = set(re.sub(r"[^\w\s]", " ", str(s2)).lower().split())
    return w1 - w2

# Get a list of word probabilities using edits with given indices
def get_word_prob(indices):
    # Count the number of appearances of each word in vandalism/constructive edits
    vandalism_words_count = defaultdict(int)
    constructive_words_count = defaultdict(int)
    
    for i in indices:
        for word in get_difference(str(train_data.loc[i, 'added_lines']), str(train_data.loc[i, 'deleted_lines'])):
            if train_data['isvandalism'][i] == True:
                vandalism_words_count[word] += 1
            else:
                constructive_words_count[word] += 1

    # Combine all unique words
    all_words = set(vandalism_words_count) | set(constructive_words_count)

    # Compute smoothed probabilities
    word_probs = {
        word: (vandalism_words_count[word] + 1) / 
              (vandalism_words_count[word] + constructive_words_count[word] + 2)
        for word in all_words
    }
    return word_probs

# Compute vandalism scores for edits with given indices using a given list of word probabilities 
def compute_vandalism_scores(word_probs, indices):
    for i in indices:
        words = get_difference(train_data['added_lines'][i], train_data['deleted_lines'][i])
        probs_of_words = [word_probs.get(w, 0.5) for w in words]

        product_p = 1
        product_1_minus_p = 1
        for p in probs_of_words:
            product_p *= p
            product_1_minus_p *= (1-p)
        
        score = product_p / (product_p + product_1_minus_p) if (product_p + product_1_minus_p) != 0 else 1
        train_data.loc[i, 'vandalism_score'] = score

In [ ]:
# Modified Stacking Classifier Method

accuracy_scores = []

for i in range(5):
    i_0 = i % 5
    i_1 = (i + 1) % 5
    i_2 = (i + 2) % 5
    i_3 = (i + 3) % 5
    i_4 = (i + 4) % 5

    word_probs = get_word_prob(fold[i_1] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_0])

    word_probs = get_word_prob(fold[i_0] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_1])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_2])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_2])
    compute_vandalism_scores(word_probs, fold[i_3])

    word_probs = get_word_prob(fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3])
    compute_vandalism_scores(word_probs, fold[i_4])

    nb = GaussianNB()
    nb.fit(train_data.loc[fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3], features], train_data.loc[fold[i_0] + fold[i_1] + fold[i_2] + fold[i_3], 'isvandalism'])
    pred = nb.predict(train_data.loc[fold[i_4], features])

    accuracy_scores.append(np.sum(pred == train_data.loc[fold[i_4], 'isvandalism'])/len(train_data.loc[fold[i_4], 'isvandalism']))
    print(confusion_matrix(train_data.loc[fold[i_4], 'isvandalism'], pred))

print(np.average(accuracy_scores))

[[1497 1117]
 [  15 2468]]
[[1550 1164]
 [  22 2362]]
[[1473 1127]
 [  18 2480]]
[[1532 1091]
 [  13 2462]]
[[1417 1123]
 [  16 2541]]
0.7761300291796965
